# Classificação com KNN e SVM

In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

print('Done')

Done


Uma vez que os dados já foram processados, a segunda parte do projeto consiste em usar modelos de aprendizado de máquina para ver se conseguimos encontrar algum atributo (tecidos) que permitam uma classificação com boa acurácia.

Mesmo que o resultado obtido seja bom ou ruim, temos que lembrar que os dados vieram de uma população diversificada em relação à idade e ao grau de esquizofrenia, assim como o tempo de acompanhamento de cada indivíduo (mais informações podem ser encontradas no link: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3358772/).

A base é dividida em treino e teste, e a partição de teste não é utilizada para treinar, de forma a evitar que o classificador seja enviesado.

Serão definidas algumas funções auxiliares para essa parte.

| Função | Descrição |
|---|---|
|<pre>prepare_dataset(conditions_to_drop, features_to_use)</pre> | Essa função recebe uma lista com as condições (classes) para remover, quando deseja-se descartar informações dos irmãos dos pacientes esquizofrênicos ou dos controles. Isso pode ser de interesse pois aumenta o número de classes para classificar, sendo que os irmãos de esquizofrênicos não necessariamente tem o mesmo transtorno, e eles possuem uma maior disposição à desenvolver o transtorno bipolar, podendo confundir o classificador. Além disso, há uma outra lista que contém os atributos para usar, quando se deseja usar mais de um ou quando queremos variar o que está sendo usado para ver como afeta a classificação |
|<pre>knn(X_train, X_test, y_train, y_test)</pre> | Recebe as partições de treino e teste e realiza o treino de um classificador KNN com número de vizinhos igual a 3, e então calcula a acurácia utilizando a partição de testes |
|<pre>svm(X_train, X_test, y_train, y_test)</pre> | Recebe as partições de treino e teste e realiza o treino de um classificador SVM com kernel linear, e então calcula a acurácia utilizando a partição de testes |

In [30]:
# Função para criar o dataset
def prepare_dataset(conditions_to_drop, features_to_use):
    
    dataset = pd.read_csv('../data/processed.csv', header=0, sep=',')

    for c in conditions_to_drop:
        dataset = dataset.drop(dataset[dataset['condition'] == c].index)
    
    X = dataset[features_to_use].values
    y = dataset['condition'].tolist()

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.20, stratify=y, shuffle=True)

    print(f'\tForam separados {len(X_train)} dados para treino, e {len(X_test)} dados para teste')
    
    return X_train, X_test, y_train, y_test


def knn(X_train, X_test, y_train, y_test):
    
    knn = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
    knn.fit(X_train, y_train)

    y_pred = knn.predict(X_test)

    print('\tAcurácia KNN:', accuracy_score(y_test, y_pred))
    
    
def svm(X_train, X_test, y_train, y_test):

    clf = SVC(gamma='auto', shrinking=True)
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    print('\tAcurácia SVM:', accuracy_score(y_test, y_pred))

In [29]:
feature = ['white-gray-relation', 'white-matter', 'gray-matter', 'csf-matter']

for f in feature:
    print(f'Classificando utilizando {f}')
    
    print('Considerando os irmãos:')
    # Considerando os irmãos
    X_train, X_test, y_train, y_test = prepare_dataset([], [f])
    knn(X_train, X_test, y_train, y_test)
    svm(X_train, X_test, y_train, y_test)

    print('Desconsiderando os irmãos')
    # Desconsiderando os irmãos
    X_train, X_test, y_train, y_test = prepare_dataset(['SCZ-SIB', 'CON-SIB'], [f])
    knn(X_train, X_test, y_train, y_test)
    svm(X_train, X_test, y_train, y_test)
    
    print('------------------------------------------------')

Classificando utilizando white-gray-relation
Considerando os irmãos:
	Foram separados 72 dados para treino, e 19 dados para teste
	Acurácia KNN: 0.21052631578947367
	Acurácia SVM: 0.3684210526315789
Desconsiderando os irmãos
	Foram separados 30 dados para treino, e 8 dados para teste
	Acurácia KNN: 0.875
	Acurácia SVM: 0.75
------------------------------------------------
Classificando utilizando white-matter
Considerando os irmãos:
	Foram separados 72 dados para treino, e 19 dados para teste
	Acurácia KNN: 0.10526315789473684
	Acurácia SVM: 0.3684210526315789
Desconsiderando os irmãos
	Foram separados 30 dados para treino, e 8 dados para teste
	Acurácia KNN: 0.625
	Acurácia SVM: 0.5
------------------------------------------------
Classificando utilizando gray-matter
Considerando os irmãos:
	Foram separados 72 dados para treino, e 19 dados para teste
	Acurácia KNN: 0.3684210526315789
	Acurácia SVM: 0.3684210526315789
Desconsiderando os irmãos
	Foram separados 30 dados para treino, e 8

## Discussão dos resultados

Pelas acuárias, podemos ver que sempre que os irmãos dos esquizofrênicos e do controle são utilizados, os dois classificadores não conseguem atingir uma acurácia maior que 0.3. Esse número pode ser baixo, mas é necessário um estudo mais aprofundado para decidir se é um bom resultado ou não (uma acurácia baixa que supera a acuária de alguns médicos já é suficiente para dizer que o modelo tem um bom desempenho, mas é preciso avaliar se esse resultado ocorreu ao acaso (utilizando um teste estatístico de p-valor)).

Quando são utilizados apenas pacientes esquizofrênicos e controle, a acuária é sempre maior ou igual que 0.5 para qualquer tecido que seja utilizado como atributo. O que apresenta maior acurácia é quando utilizamos a relação entre massa branca/massa cinzenta como atributo, obtendo uma acurácia de 0.875. Outro cuidado que temos que tomar aqui é que tanto a partição de treino quanto a de teste não são absurdamente grandes. Alguns modelos requerem uma quantidade muito grande de dados para proporcionar uma boa tarefa de predição, então apesar do alto valor, um estudo mais detalhado faz-se necessário para afirmar que os modelos realmente apresentaram um desempenho bom).

De forma geral, os modelos apresentam dados interessantes mas devemos tomar cuidado ao interpretar os dados ou tirar conclusões.